# Interview Scheduler

## Assumptions
You don't care about the ordering of the interviews, I.E. you have no preference to say "I would like to go to the UF interview before the FSU interview" assuming both were chosen. What this means - this scheduler assumes if you are going to both, you don't care about which day you go if one is on say 11/12 and the other is on 11/13. It's going to randomly pick one. This probably works for virtual interviews, but it's not very realistic for physical interviews. 

## Running
This does not support non google colab. (I mean it does....but like...just click the button)
<a href="https://colab.research.google.com/github/abaily/med_school_scheduling/blob/main/scheduler.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!git clone https://github.com/abaily/med_school_scheduling.git
%cd med_school_scheduling/

In [ ]:
#USER MODIFIED VALUES
#Blackout dates - dates you would NOT like to do interviews on for whatever reason (holiday, vacation), in format ['11/12', '11/13', '11/14']
blackout_dates = ['11/12','11/13','11/14','11/15']
#Selected schools - Schools you've already selected, with an interview date, in the format {'10/25': ['University of Florida'], '10:26': ['Florida State']}
#NOTE# the date and school name must be EXACTLY as they appear in the csv
selected_schools = {'10/25': ["University of Florida"]}

In [ ]:
import utils
import collections
import numpy as np
import random as rd 

"""Modify DEBUG to 1 if code breaks; modify EXTRA_OUTPUT to 1 if you want to see more stuff printed
"""
DEBUG = 0
EXTRA_OUTPUT = 0
"""Modify this number to run for longer/shorter. Depending how complex the problem is (how many schools you were accepted to interview for, and how many interview
days each school has) you may need to run longer. For 95% of situations, this number is more than large enough"""
GIANT_NUMBER = 10000

school_array = np.genfromtxt('School_Date_Tracker.csv', delimiter=',',names=True,dtype=None, encoding=None)

school_name_list, date_list, prio_list = utils.parse_school_array(school_array)
school_choice_dict = {name:0 for name in school_name_list}
unique_date_list = np.sort(utils.get_unique_date_list(date_list))

match_dict = {}
for date in unique_date_list:
    match_dict[date] = [row[0] for row in school_array if date in row]
match_dict.update(selected_schools)

unique_date_list = [date for date in unique_date_list if date not in blackout_dates]

In [ ]:
if EXTRA_OUTPUT:
    for key in match_dict:
        print(f'Date: {key} Schools: {match_dict[key]}')

## Here comes the limit of my technical competence. Scheduling problem was doable if I wanted to look into cplex modules for python, but uh. We didn't do that. Lot of projects going on and busted this out in a few hours, don't hate me or judge me if this gets shared lul. The following cell might take a while to run. I'd recommend if you're on google colab, you go to the edit settings, and under the runtime option, you make sure "Gpu" is selected.

In [ ]:
GIANT_NUMBER = 10 #Change this value to determine how long you want this to run for
school_value = 0
print(f'Starting School Value: {school_value}')
prio_list = np.ones(len(prio_list))

optimal_dict = {}

for i in range(GIANT_NUMBER): #lul don't judge me on how hacky this is

    school_choice_dict.update((k,0) for k in school_choice_dict)
    chosen_list = []
    comparison_dict = {}
    keys = list(match_dict.keys())
    for key in rd.sample(keys,len(keys)):
        reference_row = [value for value in match_dict[key] if value not in chosen_list]
        if len(reference_row) != 0:
            random_choice = rd.randint(0,(len(reference_row)-1))
            choice = reference_row[random_choice]
            comparison_dict[key] = choice
            chosen_list.append(choice)

    for school in list(school_choice_dict.keys()):
        if school in chosen_list:
            school_choice_dict[school] = 1
    
    iter_value = utils.get_schedule_value(prio_list, school_choice_dict)
    if iter_value > school_value: 
        print(f'School Value for iteration: {i} improved to {iter_value}')
        school_value = iter_value
        optimal_dict = collections.OrderedDict(sorted(comparison_dict.items()))
        if EXTRA_OUTPUT:
            for value in optimal_dict:
                print(f'Date: {value} School: {optimal_dict[value]}')

if DEBUG:
    print("Dates Not Used: ")
    print([value for value in unique_date_list if value not in list(optimal_dict.keys())])
    print("Schools Not Used:")
    print([value for value in school_name_list if value not in list(optimal_dict.values())])

In [ ]:
print(optimal_dict['10/25'])